# Environment setting
環境設定

In [1]:
import os
import requests
import sys
from pathlib import Path
from pprint import pprint
from typing import Optional

import pandas as pd


# determine branch, default is main
branch: str = "main"

# determine subfolder, default is None (petsard/demo/)
subfolder: Optional[str] = "use-cases"


# Check if running in Google Colab, if so, download the utils.py file from GitHub
is_colab: bool = "COLAB_GPU" in os.environ
if is_colab:
    utils_url = (
        f"https://raw.githubusercontent.com/nics-tw/petsard/{branch}/demo/utils.py"
    )
    response = requests.get(utils_url)

    if response.status_code == 200:
        with open("utils.py", "w") as f:
            f.write(response.text)

        Path("__init__.py").touch()
    else:
        raise RuntimeError(
            f"Failed to download utils.py. Status code: {response.status_code}"
        )


# If not colab, and also contains subfolderl, add the correct path of util.py
else:
    if subfolder:
        sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
# Now import and run the setup
from utils import (
    get_yaml_path,
    setup_environment,
)


setup_environment(
    is_colab,
    branch,
    benchmark_data=[
        "adult-income",
    ],
    subfolder=subfolder,
)

In [3]:
from petsard import Executor

# YAML Configuration for PETsARD
PETsARD 的 YAML 設定

## Comparing Synthesizers
比較合成演算法

In [4]:
yaml_file_case: str = "comparing-synthesizers.yaml"

yaml_path_case: str = get_yaml_path(
    is_colab=is_colab,
    yaml_file=yaml_file_case,
    branch=branch,
    subfolder=subfolder,
)

Configuration content:
---
Loader:
  data:
    filepath: 'benchmark/adult-income.csv'
Preprocessor:
  demo:
    method: 'default'
Synthesizer:
  gaussian-copula:
    method: 'sdv-single_table-gaussiancopula'
  tvae:
    method: 'sdv-single_table-tvae'
Postprocessor:
  demo:
    method: 'default'
Evaluator:
  demo-quality:
    method: 'sdmetrics-qualityreport'
Reporter:
  output:
    method: 'save_data'
    source: 'Synthesizer'
  save_report_global:
    method: 'save_report'
    granularity: 'global'
...


### Execution and Result
執行與結果

In [ ]:
# Initialize and run executor
exec_case = Executor(config=yaml_path_case)
exec_case.run()

In [ ]:
exec_case.get_result()[
    "Loader[data]_Preprocessor[demo]_Synthesizer[tvae]_Postprocessor[demo]_Evaluator[demo-quality]_Reporter[save_report_global]"
]["[global]"]

,full_expt_name,Loader,Preprocessor,Synthesizer,Postprocessor,Evaluator,Score,Column Shapes,Column Pair Trends
0,Loader[data]_Preprocessor[demo]_Synthesizer[ct...,data,demo,ctgan,demo,[global],0.704766,0.903543,0.505989
1,Loader[data]_Preprocessor[demo]_Synthesizer[ga...,data,demo,gaussian-copula,demo,[global],0.779163,0.953704,0.604621
2,Loader[data]_Preprocessor[demo]_Synthesizer[tv...,data,demo,tvae,demo,[global],0.687809,0.859874,0.515744
